In [ ]:
import os
import re
import sys
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
if '../scripts' not in sys.path:
    sys.path.append('../scripts')
from plots import plot_wmx, plot_wmx_avg, plot_w_distr, plot_weights, save_selected_w

In [ ]:
athorny_rate = 0.1
n_thorny = 8000
fmt = '../files/weights_a-thorny_rate={}_N_thorny={}.npz'
data_files = sorted(glob.glob(fmt.format('*', n_thorny)))
data = np.load(data_files[0], allow_pickle=True)
weights = data['weights'].item()
config = data['config'].item()
connections = config['connectivity']

In [ ]:
wgts = weights['thorny']['thorny'].todense()
fig,ax = plt.subplots(1, 2, figsize=(10,5))
plot_wmx(wgts, ax=ax[0])
plot_wmx_avg(wgts, n_pops=100, ax=ax[1])

In [ ]:
pre, post = 'thorny', 'a-thorny'
W = []
rate = []
for data_file in data_files:
    data = np.load(data_file, allow_pickle=True)
    weights = data['weights'].item()
    W.append(weights[pre][post].todense())
    rate.append(float(re.findall('rate=\d+.\d+', data_file)[0].split('=')[1]))
idx = [a == pre and b == post for a,b in zip(connections['pre'], connections['post'])].index(True)
n_pre, n_post = W[0].shape
middle = np.linspace(0, n_pre, n_post+2)[1:-1]
sigma = connections['n_pre'][idx]

In [ ]:
fig,ax = plt.subplots(1, 1)
cmap = plt.get_cmap('cividis', len(W))
for i,wgts in enumerate(W):
    ax.plot(wgts * 1e9, color=cmap(i), lw=1, label=f'A-th rate: {rate[i]} sp/s')
ax.set_xlim([middle-3*sigma, middle+3*sigma])
ax.legend(loc='best', frameon=False)
ax.set_xlabel('Presynaptic neuron ID')
ax.set_ylabel('Synaptic weight (nS)')
for side in 'right','top':
    ax.spines[side].set_visible(False)
fig.tight_layout()